In [1]:
import os
import re
import tempfile
import time
import zipfile

import contractions
import nltk
import pandas as pd
import requests
from IPython import get_ipython
from IPython.display import display
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

pd.set_option('max_colwidth', 400)

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

t = time.time()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Préprocessing du texte

accents, contractions, lowercase, newlines, lemmatization, stemming

In [2]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [3]:
NEGATION_SET = {'no', 'not'}

In [4]:
def negate_sequence(word_list):
    if len(word_list) > 1:
        if word_list[0] in NEGATION_SET:
            return [f"not_{word_list[1]}"] + negate_sequence(word_list[2:])
        else:
            return [word_list[0]] + negate_sequence(word_list[1:])
    else:
        return word_list

In [5]:
word_list = ["Hello", "not", "yellow", "bad", "not", "not", "bad"]
negate_sequence(word_list)

['Hello', 'not_yellow', 'bad', 'not_not', 'bad']

In [6]:
def text_preprocessing(serie, stop_words, lemmatization, negation):
    # lowercase
    serie = serie.map(lambda x: x.lower())   

    # remove extra newlines
    serie = serie.map(lambda x: re.sub(r'[\r|\n|\r\n]+', ' ', x))

    # remove @tag
    serie = serie.map(lambda x: re.sub(r'@[\S]+', '', x))

    # remove URL
    serie = serie.map(lambda x: re.sub('https?://[\S]+', '', x))

    # remove contractions
    serie = serie.map(lambda x: contractions.fix(x).lower())

    # remove hashtag and numbers
    serie = serie.map(lambda x: re.sub("[^a-zA-Z]", " ", x))

    # tokenization
    serie = serie.map(word_tokenize)

    if stop_words:        
        # remove stop words
        stop_words = set(stopwords.words('english')).difference(NEGATION_SET)
        serie = serie.map(lambda x: [word for word in x if word not in stop_words])
    
    if lemmatization:
        # lemmatization    
        serie = serie.map(nltk.tag.pos_tag)
        serie = serie.map(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
        wordnet_lemmatizer = WordNetLemmatizer()
        serie = serie.map(lambda x: [wordnet_lemmatizer.lemmatize(word, tag) for (word, tag) in x])
    
    if negation:
        serie = serie.map(lambda x: negate_sequence(x))

    serie = serie.map(lambda x: ' '.join(word for word in x))

    return serie

In [7]:
text = "i agreed. this was useful. democrats should definitely not vote for joe biden in the primary."
for stop_words in [False, True]:
    for lemmatization in [False, True]:
        for negation in [False, True]:
            print(text_preprocessing(pd.Series([text]), stop_words=stop_words, lemmatization=lemmatization, negation=negation))

0    i agreed this was useful democrats should definitely not vote for joe biden in the primary
dtype: object
0    i agreed this was useful democrats should definitely not_vote for joe biden in the primary
dtype: object
0    i agree this be useful democrat should definitely not vote for joe biden in the primary
dtype: object
0    i agree this be useful democrat should definitely not_vote for joe biden in the primary
dtype: object
0    agreed useful democrats definitely not vote joe biden primary
dtype: object
0    agreed useful democrats definitely not_vote joe biden primary
dtype: object
0    agree useful democrat definitely not vote joe biden primary
dtype: object
0    agree useful democrat definitely not_vote joe biden primary
dtype: object


# 2. Sentiment140
## 2.1. Téléchargement de la base Sentiment140

In [8]:
url, destname = 'http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip', 'sentiment140'
temporary_location = "temp"

def download_unzip(url, dirname=tempfile.gettempdir(), destname="file"):
    myfile = requests.get(url)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    open(os.path.join(dirname, destname + '.zip'), 'wb').write(myfile.content)
    with zipfile.ZipFile(os.path.join(dirname, destname + '.zip'), 'r') as zip_ref:
        zip_ref.extractall(os.path.join(dirname, destname))

In [9]:
download_unzip(url, dirname=temporary_location, destname=destname)

In [10]:
trainfile = os.path.join(temporary_location, destname, "training.1600000.processed.noemoticon.csv")
# testfile = os.path.join(temporary_location, destname, "testdata.manual.2009.06.14.csv")
columns = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']

In [11]:
# Import train data set
df = pd.read_csv(trainfile,
                 header=None,
                 names=columns,
                 encoding='latin-1')
# df = df.head(1000)
df

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur


## 2.2. Préprocessing Sentiment140

In [12]:
df.sentiment = df.sentiment.apply(lambda x: 'Negative' if x == 0 else 'Positive')

In [13]:
df[['text']].head(10)

,text
0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
5,@Kwesidei not the whole crew
6,Need a hug
7,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?"
8,@Tatiana_K nope they didn't have it
9,@twittera que me muera ?


In [14]:
%%time
for stop_words in [False, True]:
    for lemmatization in [False, True]:
        for negation in [False, True]:
            dfcopy = df.copy()
            a = time.time()
            dfcopy.text = text_preprocessing(dfcopy.text, stop_words=stop_words, lemmatization=lemmatization, negation=negation)
            dfcopy = dfcopy[dfcopy.text != '']

            print(f"stop_words: {stop_words}, lemmatization: {lemmatization}, negation: {negation}, time: {time.time() - a}")
            display(dfcopy[['text']].head(10))

            file = "train"
            if stop_words:
                file += "_stop"
            if lemmatization:
                file += "_lemm"
            if negation:
                file += "_neg"
            dfcopy.to_pickle(os.path.join("data", "sentiment140", file+".bz2"))

stop_words: False, lemmatization: True, negation: False, time: 1858.179282426834


,text
0,awww that be a bummer you shoulda get david carr of third day to do it d
1,be upset that he can not update his facebook by texting it and might cry a a result school today also blah
2,i dive many time for the ball manage to save the rest go out of bound
3,my whole body feel itchy and like it on fire
4,no it be not behave at all i be mad why be i here because i can not see you all over there
5,not the whole crew
6,need a hug
7,hey long time no see yes rain a bit only a bit lol i be fine thanks how be you
8,nope they do not have it
9,que me muera


stop_words: False, lemmatization: True, negation: True, time: 2001.8654296398163


,text
0,awww that be a bummer you shoulda get david carr of third day to do it d
1,be upset that he can not_update his facebook by texting it and might cry a a result school today also blah
2,i dive many time for the ball manage to save the rest go out of bound
3,my whole body feel itchy and like it on fire
4,not_it be not_behave at all i be mad why be i here because i can not_see you all over there
5,not_the whole crew
6,need a hug
7,hey long time not_see yes rain a bit only a bit lol i be fine thanks how be you
8,nope they do not_have it
9,que me muera


stop_words: True, lemmatization: True, negation: False, time: 1608.4383187294006


,text
0,awww bummer shoulda get david carr third day
1,upset not update facebook texting might cry result school today also blah
2,dive many time ball manage save rest go bound
3,whole body feel itchy like fire
4,no not behave mad not see
5,not whole crew
6,need hug
7,hey long time no see yes rain bite bit lol fine thanks
8,nope not
9,que muera


stop_words: True, lemmatization: True, negation: True, time: 1465.399047613144


,text
0,awww bummer shoulda get david carr third day
1,upset not_update facebook texting might cry result school today also blah
2,dive many time ball manage save rest go bound
3,whole body feel itchy like fire
4,not_not behave mad not_see
5,not_whole crew
6,need hug
7,hey long time not_see yes rain bite bit lol fine thanks
8,nope not
9,que muera


Wall time: 1h 56min 39s


# 3. Base Twitter webscrappée
## 3.1 Chargement de la base

In [14]:
df = pd.read_pickle(os.path.join("data", "web", "web_parse_2019.bz2"))
df

,search,author,time,url,text,lang,reply
0,biden,Maggie Klaus 🤶🏻,2019-01-01 22:37:46,/Maggie_Klaus/status/1080231636334067712,Shut the f*ck up about Hillary’s likability. When you spew shit like that you give into the lies that Biden and Bernie tried to spread. That she didn’t inspire or bring joy. More votes than both those mediocre white men. More votes than any white man in history. #StillWithHer,en,False
1,biden,The Salt Lake Tribune,2019-01-01 22:15:18,/sltrib/status/1080225983083487232,"The @UUtah agreed to pay former Vice President @JoeBiden $100,000 to speak on campus last month, but when Biden heard the money would come from state coffers he declined the check.",en,False
2,biden,The New York Times,2019-01-01 20:00:08,/nytimes/status/1080191967873716224,"A series of careful financial decisions, and the creation of nonprofits and academic centers staffed by close advisers: How Joe Biden has paved the way for a possible presidential run",en,False
3,biden,Matt Binder,2019-01-01 22:18:51,/MattBinder/status/1080226876226965504,i agree. this is useful. democrats should definitely not vote for joe biden in the primary.,en,False
4,biden,Patrick Healy,2019-01-01 19:58:58,/patrickhealynyt/status/1080191673701933059,NEWS: Joe Biden's 2020 campaign-in-waiting includes longtime advisers making six figures at nonprofits & academic centers he started. He himself has tried to balance making $$ & political viability. Great details by ⁦@ksacknyt⁩ ⁦@alexburnsNYT⁩ https://nyti.ms/2GOiMJf?smid=nytcore-ios-share…,en,False
...,...,...,...,...,...,...,...
10689,trump,VioletDaCat@danabennett,2019-12-27 23:50:58,/VioletDaCatdan1/status/1210709687466741761,"Seriously, when is it going to be over the top for Mr. Dorsey??? This shit Trump is saying is Against. The. Law. Period.",en,False
10690,trump,Isabella aka #PumpkinSpiceElf,2019-12-27 23:59:03,/Isabella040408/status/1210711722488729601,"Yes, I had a feeling Trump was really talking about Tom Hanks.\n#Justiceforthechildren #WWG1WGA_WORLDWIDE",en,False
10691,trump,Teresae[Globe terrestre Europe-Afrique],2019-12-27 23:52:18,/teresa3llen/status/1210710026077106176,"The producers of the movie had to agree to a cameo in order to film in the hotel, which Trump at the time owned.",en,False
10692,trump,Truth Quest,2019-12-27 23:56:12,/TheTruthQuester/status/1210711004570632192,He gave Trump the presidency and now he is profiting from it,en,False


## 3.2 Préprocessing de la base

On supprime les doublons éventuels :

In [15]:
df.drop_duplicates(subset="url", keep='first', inplace=True)
df.shape

(10608, 7)

In [16]:
def filtre(text):
    lower_text = text.lower()
    if "trumps" in lower_text:
        if "trump" in lower_text.replace("trumps", ""):
            return True
        else:
            return False
    else:
        return True

La recherche du mot `trump` dans Twitter fournit aussi des tweets comprenants le mot `trumps` (issu du nom ou verbe `trump`), on supprime donc ces tweets.

In [17]:
df = df[(df.search == 'biden') | (df.search == "trump") & (df.text.apply(filtre))]
df.shape

(10588, 7)

In [18]:
df[['text']].head(10)

,text
0,Shut the f*ck up about Hillary’s likability. When you spew shit like that you give into the lies that Biden and Bernie tried to spread. That she didn’t inspire or bring joy. More votes than both those mediocre white men. More votes than any white man in history. #StillWithHer
1,"The @UUtah agreed to pay former Vice President @JoeBiden $100,000 to speak on campus last month, but when Biden heard the money would come from state coffers he declined the check."
2,"A series of careful financial decisions, and the creation of nonprofits and academic centers staffed by close advisers: How Joe Biden has paved the way for a possible presidential run"
3,i agree. this is useful. democrats should definitely not vote for joe biden in the primary.
4,NEWS: Joe Biden's 2020 campaign-in-waiting includes longtime advisers making six figures at nonprofits & academic centers he started. He himself has tried to balance making $$ & political viability. Great details by ⁦@ksacknyt⁩ ⁦@alexburnsNYT⁩ https://nyti.ms/2GOiMJf?smid=nytcore-ios-share…
5,"Also on Fox News, Newt Gingrich praised Sen. Marco Rubio for being “very likable.”"
6,"Republicans at Fox News even had high praise for Vice President Joe Biden’s likability in 2015. “One of the most likable people in Washington,” Dana Perino said."
7,for every joe biden he votes for im going to not vote for ten
8,The unspoken and untrue premise of this piece is that people like Biden or Bernie are likeable at all.
9,"Re the 2020 Dem race:\n- I’ll vote for whoever is the nom. A given.\n- I really don’t want Biden. Some say he’s the safe bet. I’m not so sure.\n- A woman, please? PLEASE?\n- That said, it breaks my ❤️ that Hillary will not be there. She earned the office but was ROBBED.\n- NO BERNIE."


In [19]:
for stop_words in [False, True]:
    for lemmatization in [False, True]:
        for negation in [False, True]:
            dfcopy = df.copy()
            a = time.time()
            dfcopy.text = text_preprocessing(dfcopy.text, stop_words=stop_words, lemmatization=lemmatization, negation=negation)
            dfcopy = dfcopy[dfcopy.text != '']

            print(f"stop_words: {stop_words}, lemmatization: {lemmatization}, negation: {negation}, time: {time.time() - a}")
            display(dfcopy[['text']].head(10))

            file = "web"
            if stop_words:
                file += "_stop"
            if lemmatization:
                file += "_lemm"
            if negation:
                file += "_neg"
            dfcopy.to_pickle(os.path.join("data", "web", file + ".bz2"))

stop_words: False, lemmatization: False, negation: False, time: 1.7353599071502686


,text
0,shut the f ck up about hillary s likability when you spew shit like that you give into the lies that biden and bernie tried to spread that she did not inspire or bring joy more votes than both those mediocre white men more votes than any white man in history stillwithher
1,the agreed to pay former vice president to speak on campus last month but when biden heard the money would come from state coffers he declined the check
2,a series of careful financial decisions and the creation of nonprofits and academic centers staffed by close advisers how joe biden has paved the way for a possible presidential run
3,i agree this is useful democrats should definitely not vote for joe biden in the primary
4,news joe biden s campaign in waiting includes longtime advisers making six figures at nonprofits academic centers he started he himself has tried to balance making political viability great details by
5,also on fox news newt gingrich praised sen marco rubio for being very likable
6,republicans at fox news even had high praise for vice president joe biden s likability in one of the most likable people in washington dana perino said
7,for every joe biden he votes for i am going to not vote for ten
8,the unspoken and untrue premise of this piece is that people like biden or bernie are likeable at all
9,re the dem race i will vote for whoever is the nom a given i really do not want biden some say he is the safe bet i am not so sure a woman please please that said it breaks my that hillary will not be there she earned the office but was robbed no bernie


stop_words: False, lemmatization: False, negation: True, time: 1.989685297012329


,text
0,shut the f ck up about hillary s likability when you spew shit like that you give into the lies that biden and bernie tried to spread that she did not_inspire or bring joy more votes than both those mediocre white men more votes than any white man in history stillwithher
1,the agreed to pay former vice president to speak on campus last month but when biden heard the money would come from state coffers he declined the check
2,a series of careful financial decisions and the creation of nonprofits and academic centers staffed by close advisers how joe biden has paved the way for a possible presidential run
3,i agree this is useful democrats should definitely not_vote for joe biden in the primary
4,news joe biden s campaign in waiting includes longtime advisers making six figures at nonprofits academic centers he started he himself has tried to balance making political viability great details by
5,also on fox news newt gingrich praised sen marco rubio for being very likable
6,republicans at fox news even had high praise for vice president joe biden s likability in one of the most likable people in washington dana perino said
7,for every joe biden he votes for i am going to not_vote for ten
8,the unspoken and untrue premise of this piece is that people like biden or bernie are likeable at all
9,re the dem race i will vote for whoever is the nom a given i really do not_want biden some say he is the safe bet i am not_so sure a woman please please that said it breaks my that hillary will not_be there she earned the office but was robbed not_bernie


stop_words: False, lemmatization: True, negation: False, time: 19.112591981887817


...

,text
0,shut f ck hillary likability spew shit like give lies biden bernie tried spread not_inspire bring joy votes mediocre white men votes white man history stillwithher
1,agreed pay former vice president speak campus last month biden heard money would come state coffers declined check
2,series careful financial decisions creation nonprofits academic centers staffed close advisers joe biden paved way possible presidential run
3,agree useful democrats definitely not_vote joe biden primary
4,news joe biden campaign waiting includes longtime advisers making six figures nonprofits academic centers started tried balance making political viability great details
5,also fox news newt gingrich praised sen marco rubio likable
6,republicans fox news even high praise vice president joe biden likability one likable people washington dana perino said
7,every joe biden votes going not_vote ten
8,unspoken untrue premise piece people like biden bernie likeable
9,dem race vote whoever nom given really not_want biden say safe bet not_sure woman please please said breaks hillary not_earned office robbed not_bernie


stop_words: True, lemmatization: True, negation: False, time: 16.029897451400757


,text
0,shut f ck hillary likability spew shit like give lie biden bernie try spread not inspire bring joy vote mediocre white men vote white man history stillwithher
1,agree pay former vice president speak campus last month biden heard money would come state coffer decline check
2,series careful financial decision creation nonprofit academic center staff close adviser joe biden pave way possible presidential run
3,agree useful democrat definitely not vote joe biden primary
4,news joe biden campaign wait include longtime adviser make six figure nonprofits academic center start tried balance make political viability great detail
5,also fox news newt gingrich praise sen marco rubio likable
6,republican fox news even high praise vice president joe biden likability one likable people washington dana perino say
7,every joe biden vote go not vote ten
8,unspoken untrue premise piece people like biden bernie likeable
9,dem race vote whoever nom give really not want biden say safe bet not sure woman please please say break hillary not earn office rob no bernie


stop_words: True, lemmatization: True, negation: True, time: 15.159830808639526


,text
0,shut f ck hillary likability spew shit like give lie biden bernie try spread not_inspire bring joy vote mediocre white men vote white man history stillwithher
1,agree pay former vice president speak campus last month biden heard money would come state coffer decline check
2,series careful financial decision creation nonprofit academic center staff close adviser joe biden pave way possible presidential run
3,agree useful democrat definitely not_vote joe biden primary
4,news joe biden campaign wait include longtime adviser make six figure nonprofits academic center start tried balance make political viability great detail
5,also fox news newt gingrich praise sen marco rubio likable
6,republican fox news even high praise vice president joe biden likability one likable people washington dana perino say
7,every joe biden vote go not_vote ten
8,unspoken untrue premise piece people like biden bernie likeable
9,dem race vote whoever nom give really not_want biden say safe bet not_sure woman please please say break hillary not_earn office rob not_bernie


In [ ]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")